In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np

In [ ]:
from nflows.flows import MaskedAutoregressiveFlow

from counterfactuals.datasets import (
    AdultDataset,
    CompasDataset,
)
from counterfactuals.discriminative_models import LogisticRegression
from counterfactuals.optimizers.base import BaseCounterfactualModel

from counterfactuals.metrics.metrics import (
    evaluate_cf,
)

# from sklearn.linear_model import LogisticRegression
# from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Create dataset

In [ ]:
dataset = CompasDataset(file_path="../data/compas_two_years.csv")
dataset = AdultDataset(file_path="../data/adult.csv")

In [ ]:
# disc_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=100)

# from sklearn.linear_model import LogisticRegression

# disc_model = LogisticRegression()
# disc_model.fit(dataset.X_train, dataset.y_train.reshape(-1))
# print(classification_report(dataset.y_test, disc_model.predict(dataset.X_test)))
# print(disc_model.predict(dataset.X_test).shape)

# build custom module for logistic regression
# class LogisticRegression(torch.nn.Module):
#     # build the constructor
#     def __init__(self, n_inputs, n_outputs):
#         super(LogisticRegression, self).__init__()
#         self.linear = torch.nn.Linear(n_inputs, n_outputs)
#     # make predictions
#     def forward(self, x):
#         y_pred = torch.sigmoid(self.linear(x))
#         return y_pred
#     def fit(self, train_loader):
#         # defining the optimizer
#         optimizer = torch.optim.Adam(self.linear.parameters(), lr=0.01)
#         # defining Cross-Entropy loss
#         criterion = torch.nn.BCELoss()

#         epochs = 200

#         for epoch in range(epochs):
#             for i, (examples, labels) in enumerate(train_loader):
#                 optimizer.zero_grad()
#                 outputs = self.forward(examples)
#                 labels = labels.reshape(-1, 1)
#                 loss = criterion(outputs, labels)
#                 # Loss.append(loss.item())
#                 loss.backward()
#                 optimizer.step()

#     def predict(self, X_test):
#         probs = self.forward(torch.from_numpy(X_test))
#         probs = probs > 0.5
#         return np.squeeze(probs.numpy().astype(np.float32))


disc_model = LogisticRegression(dataset.X_train.shape[1], 1)
train_dataloader = dataset.train_dataloader(
    batch_size=128, shuffle=True, noise_lvl=1e-5
)
disc_model.fit(train_dataloader)
print(classification_report(dataset.y_test, disc_model.predict(dataset.X_test)))
disc_model.predict(dataset.X_test).shape

# Relabeling

In [ ]:
y_pred_train = disc_model.predict(dataset.X_train)
y_pred_test = disc_model.predict(dataset.X_test)
dataset.y_train = y_pred_train
dataset.y_test = y_pred_test

# noise_lvl - zaszumianie numerycznych cech treningowego datasetu
train_dataloader = dataset.train_dataloader(
    batch_size=128, shuffle=True, noise_lvl=1e-5
)
test_dataloader = dataset.test_dataloader(batch_size=128, shuffle=False)

# Create flow model

In [ ]:
# from nflows.flows import SimpleRealNVP

# flow = SimpleRealNVP(use_volume_preserving=True, features=dataset.X_train.shape[1], hidden_features=4, context_features=1, num_layers=5)

flow = MaskedAutoregressiveFlow(
    features=dataset.X_train.shape[1],
    hidden_features=4,
    num_blocks_per_layer=2,
    num_layers=1,
    context_features=1,
)

# Define custom search step within class

In [ ]:
class CustomApproach(BaseCounterfactualModel):
    def search_step(
        self, x_param, x_origin, context_origin, context_target, **search_step_kwargs
    ):
        alpha = search_step_kwargs.get("alpha", None)
        beta = search_step_kwargs.get("beta", None)
        if alpha is None:
            raise ValueError("Parameter 'alpha' should be in kwargs")
        if beta is None:
            raise ValueError("Parameter 'beta' should be in kwargs")

        dist = torch.linalg.norm(x_origin - x_param, axis=1)
        self.lr_model.eval()
        outputs = self.lr_model.forward(x_param)
        loss_d = self.criterion(outputs, context_target)

        p_x_param_c_orig = self.gen_model.log_prob(x_param, context=context_origin)
        p_x_param_c_target = self.gen_model.log_prob(x_param, context=context_target)
        p_x_orig_c_orig = self.gen_model.log_prob(
            x_origin, context=context_origin.flatten()[0].repeat((x_origin.shape[0], 1))
        )

        p_x_param_c_orig_with_beta = p_x_param_c_orig + beta
        max_inner = torch.nn.functional.relu(p_x_orig_c_orig - p_x_param_c_target)
        max_outer = torch.nn.functional.relu(
            p_x_param_c_orig_with_beta - p_x_param_c_target
        )
        loss = dist + alpha * (max_outer + max_inner + loss_d)
        return loss, dist, max_inner, max_outer

# Create cf class, train and test flow model

In [ ]:
cf = CustomApproach(
    gen_model=flow, disc_model=disc_model, checkpoint_path="model.pt", neptune_run=None
)

In [ ]:
cf.lr_model = disc_model
cf.criterion = torch.nn.BCELoss()
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=100,
    patience=20,
    eps=1e-3,  # eps for patience
)

In [ ]:
cf.test_model(test_loader=test_dataloader)

# Search counterfactuals

In [ ]:
search_step_kwargs = {
    "alpha": 20,
    "beta": 0.1,
}
test_dataloader = dataset.test_dataloader(batch_size=16, shuffle=False)
Xs_cf, Xs_orig, ys_orig = cf.search_batch(
    dataloader=test_dataloader, epochs=1000, lr=0.005, **search_step_kwargs
)

# Evaluate

In [ ]:
evaluate_cf(
    cf_class=None,
    disc_model=disc_model,
    X=Xs_orig,
    X_cf=Xs_cf,
    model_returned=np.ones(Xs_cf.shape[0]).astype(bool),
    continuous_features=dataset.numerical_features,
    categorical_features=dataset.categorical_features,
    X_train=dataset.X_train,
    y_train=dataset.y_train,
    X_test=dataset.X_test,
    y_test=dataset.y_test,
)